# 서울 소방서 정보 추출 및 지도에 표시

In [1]:
from bs4 import BeautifulSoup
import pandas as pd
import time
from selenium import webdriver
from selenium.webdriver.common.keys import Keys
from selenium.webdriver.common.by import By
import folium
from urllib.parse import quote
import json
import requests

In [41]:
# kakao key 불러오기
with open('C:\workspace\kakao_key.txt') as file:
    kakao_key = file.read()

## 1. 서울 검색

In [8]:
url = 'https://www.nfa.go.kr/nfa/introduce/status/firestationidfo/?searchDistance=10&searchMode=keyword&myX=37.5605672&myY=126.9433486&searchKeyword='
driver = webdriver.Chrome('chromedriver.exe')
driver.get(url)

C:\Users\tjoeun\AppData\Local\Temp\ipykernel_2856\1194244807.py:2: DeprecationWarning: executable_path has been deprecated, please pass in a Service object
  driver = webdriver.Chrome('chromedriver.exe')


In [40]:
# '서울' 검색
driver.find_element(By.ID, 'searchKeyword').send_keys('서울')
driver.find_element(By.ID, 'fsSearchBtn').click()
time.sleep(3)

# 2. 1페이지 정보만 가져오기

In [41]:
soup = BeautifulSoup(driver.page_source, 'html.parser')

In [44]:
lis = soup.select('.stations-list li')

In [45]:
li = lis[0]

In [47]:
# 소방서 이름
title = li.select_one('.title').text
title

'동작소방서'

In [49]:
# 소방서 주소
addr = li.select_one('address').text
addr

'서울특별시 동작구 여의대방로16길 55(신대방동)'

In [50]:
# 소방서 전화번호
tel = li.select_one('.tel').text
tel

'02-847-1190'

In [52]:
lines = []

for li in lis:
    title = li.select_one('.title').text
    addr = li.select_one('address').text
    tel = li.select_one('.tel').text

    lines.append([title, addr, tel])

In [53]:
df = pd.DataFrame(lines, columns=['관서명', '주소', '전화번호'])
df

,관서명,주소,전화번호,위도,경도
0,동작소방서,서울특별시 동작구 여의대방로16길 55(신대방동),02-847-1190,37.49461824137466,126.91769928732266
1,서대문소방서,서울특별시 서대문구 연희로 182(연희동),02-3144-1190,37.5732220853166,126.936028277351
2,광진소방서,서울특별시 광진구 광나루로 480(구의동),02-457-0119,37.5448217947148,127.082767763793
3,송파소방서,서울특별시 송파구 오금로51길 56(마천동),02-403-2119,37.4999307947939,127.142637507507
4,양천소방서,서울특별시 양천구 목동서로 180(목동),02-2655-1119,37.53017178771259,126.87206679012293
5,은평소방서,서울특별시 은평구 통일로 962(진관동),02-355-0119,37.628911419546,126.919821655113
6,노원소방서,서울특별시 노원구 한글비석로 1길 8(하계동),02-977-0119,37.63814511542415,127.07112890609659
7,종로소방서,서울특별시 종로구 종로1길 28(수송동),02-735-6119,37.57693645990929,126.98770584209309
8,도봉소방서,서울특별시 도봉구 도봉로 666(방학동),02-3492-3437,37.664116018701,127.043054892229
9,용산소방서,서울특별시 용산구 한강대로 167(한강로2가),02-794-0119,37.534116454306165,126.97151106694338


# 3. 전체 페이지 가져오기

In [74]:
url = 'https://www.nfa.go.kr/nfa/introduce/status/firestationidfo/?searchDistance=10&searchMode=keyword&myX=37.5605672&myY=126.9433486&searchKeyword='
driver = webdriver.Chrome('chromedriver.exe')
driver.get(url)
driver.find_element(By.ID, 'searchKeyword').send_keys('서울')
driver.find_element(By.ID, 'fsSearchBtn').click()
time.sleep(3)

C:\Users\tjoeun\AppData\Local\Temp\ipykernel_2856\3458345679.py:2: DeprecationWarning: executable_path has been deprecated, please pass in a Service object
  driver = webdriver.Chrome('chromedriver.exe')


In [56]:
# 검색결과 건수 가져오기
soup = BeautifulSoup(driver.page_source, 'html.parser')
strong = soup.select('.stations-local-after > p > strong')[1].string
count = int(strong[0:-1])/10
count = int(count)
count

10

In [75]:
lines = []
for i in range(count):
    # 페이지 이동
    if i >= 1 and i % 2 == 0: 
        driver.find_element(By.CSS_SELECTOR, '.next_page').click()
        time.sleep(3)
    if i >= 1 and i % 2 == 1:
        driver.find_element(By.XPATH, '//*[@id="listForm"]/div/section/ul/li[1]/div/div/ul/li[4]/a').click()
        time.sleep(3)
    
    # 각 페이지의 데이터 추출
    soup = BeautifulSoup(driver.page_source, 'html.parser')
    divs = soup.select('.stations-list li')
    
    for div in divs:
        title = div.select_one('.title').text
        addr = div.select_one('address').text
        tel = div.select_one('.tel').text
        
        lines.append([title, addr, tel])

In [76]:
df = pd.DataFrame(lines, columns=['관서명', '주소', '전화번호'])
df

,관서명,주소,전화번호
0,동작소방서,서울특별시 동작구 여의대방로16길 55(신대방동),02-847-1190
1,서대문소방서,서울특별시 서대문구 연희로 182(연희동),02-3144-1190
2,광진소방서,서울특별시 광진구 광나루로 480(구의동),02-457-0119
3,송파소방서,서울특별시 송파구 오금로51길 56(마천동),02-403-2119
4,양천소방서,서울특별시 양천구 목동서로 180(목동),02-2655-1119
...,...,...,...
95,송파소방서-운동장-119 안전센터,서울특별시 송파구 올림픽로 25 (잠실동),02-2203-2380
96,송파소방서-잠실-119 안전센터,서울특별시 송파구 석촌호수로 151 (잠실동),02-422-0119
97,송파소방서-방이-119 안전센터,서울특별시 송파구 강동대로 286 (방이동),02-409-0059
98,송파소방서-거여-119 안전센터,서울특별시 송파구 마천로 329 (마천동),02-400-0119


# 4. 위도, 경도 추가하기

In [84]:
# kakao_key를 통해서 모든 소방청 위도 경도 좌표 구하기
local_url = 'https://dapi.kakao.com/v2/local/search/address.json'
header = {"Authorization": "KakaoAK " + kakao_key}

lngs, lats = [], []
for i in df.index:
    url = f'{local_url}?query={quote(df.주소[i])}'
    result = requests.get(url, headers = headers).json()['documents']
    try:
        lngs.append(float(result[0]['x']))
        lats.append(float(result[0]['y']))
    except:
        print(df.관서명[i])

In [80]:
# '은평소방서-수색-119 안전센터' 주소를 확인
df[df.관서명.isin(['은평소방서-수색-119 안전센터'])].주소

30    서울특별시 은평구 수색로 294 (수색동)
Name: 주소, dtype: object

In [81]:
# kakao에 주소가 있는지 확인
url = f"{local_url}?query={quote('서울특별시 은평구 수색로 294')}"
result = requests.get(url, headers=header).json()
result

{'documents': [],
 'meta': {'is_end': True, 'pageable_count': 0, 'total_count': 0}}

In [82]:
# 비슷한 위치의 다른 소방서 주소로 변경
df.주소[30] = "서울특별시 은평구 통일로 962"

In [85]:
local_url = 'https://dapi.kakao.com/v2/local/search/address.json'
header = {"Authorization": "KakaoAK " + kakao_key}

lngs, lats = [], []
for i in df.index:
    url = f'{local_url}?query={quote(df.주소[i])}'
    result = requests.get(url, headers = headers).json()['documents']
    try:
        lngs.append(float(result[0]['x']))
        lats.append(float(result[0]['y']))
    except:
        print(df.관서명[i])

In [94]:
df['위도'] = lats
df['경도'] = lngs
df

,관서명,주소,전화번호,위도,경도
0,동작소방서,서울특별시 동작구 여의대방로16길 55(신대방동),02-847-1190,37.494672,126.917719
1,서대문소방서,서울특별시 서대문구 연희로 182(연희동),02-3144-1190,37.573205,126.935996
2,광진소방서,서울특별시 광진구 광나루로 480(구의동),02-457-0119,37.544826,127.082779
3,송파소방서,서울특별시 송파구 오금로51길 56(마천동),02-403-2119,37.499833,127.142592
4,양천소방서,서울특별시 양천구 목동서로 180(목동),02-2655-1119,37.530236,126.872342
...,...,...,...,...,...
95,송파소방서-운동장-119 안전센터,서울특별시 송파구 올림픽로 25 (잠실동),02-2203-2380,37.516200,127.075940
96,송파소방서-잠실-119 안전센터,서울특별시 송파구 석촌호수로 151 (잠실동),02-422-0119,37.507316,127.093893
97,송파소방서-방이-119 안전센터,서울특별시 송파구 강동대로 286 (방이동),02-409-0059,37.519785,127.136572
98,송파소방서-거여-119 안전센터,서울특별시 송파구 마천로 329 (마천동),02-400-0119,37.494163,127.151828


In [93]:
df.to_csv('서울시 소방서.csv', index=False)

# 5. 지도에 표시하기

In [105]:
map = folium.Map(location=[df.위도.mean(), df.경도.mean()], zoom_start=12)
for i in df.index[:]:
    folium.Marker(
        location=[df.위도[i],df.경도[i]], 
        popup=folium.Popup(df.주소[i], max_width=200), 
        tooltip=df.관서명[i],
        icon = folium.Icon(icon = 'glyphicon glyphicon-home')
    ).add_to(map)
title = '<h3 align="center" style="font-size:20px">서울 소재 소방서위치</h3>'
map.get_root().html.add_child(folium.Element(title))
map